<h1 align = "center"> Spark DataFrames </h1>

Ověření inicializace Spark contextu.

In [ ]:
sc.version

Vytvoření SparkSession.

# Vytvoření DataFramu

Základní nastavení

Vytvoření dataframu z RDD

Vytvoření DataFramu.

Vytvoření Datasetu.

Porovnání rozdílu DataFramu a Datasetu

## Práce s DataFramem

### SELECT

### WHERE

### ORDER BY

### GROUP BY

# Samostatná práce

In [ ]:
val spark = SparkSession
      .builder
      .appName("SparkSQL")
      .master("local[*]")
      .getOrCreate()

In [ ]:
case class FakeFriends(id: Int, name: String, age: Int, friends: Long)

Vytvoření nového Datasetu z souboru fakefriends.csv.

Vypsat schéma nového Datasetu.  

Vypsání dat v Datasetu.

Vybrat uživatele starší 30 let.

Seřadit podle věku a sestupně vypsat výsledek.